<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Сравнение-моделей" data-toc-modified-id="Сравнение-моделей-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Сравнение моделей</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Нужно построить модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

import sys
!{sys.executable} -m spacy download en_core_web_sm

In [236]:
import pandas
import numpy as np
import nltk
import re 
import string
import spacy

from tqdm import notebook 

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from scipy.sparse import vstack

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

In [237]:
comments = pandas.read_csv('/datasets/toxic_comments.csv')

In [238]:
comments

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when your view completely contradicts the coverage in reliable sources, why should anyone care what you feel? You can't even give a consistent argument - is the opening only supposed to mention significant aspects, or the """"most significant"""" ones? \n\n""",0
159288,159447,You should be ashamed of yourself \n\nThat is a horrible thing you put on my talk page. 128.61.19.93,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for prostitution ring. - Crunch Captain.",0
159290,159449,And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.,0


Есть столбец Unnamed:0, который, видимо, появился по ошибке, его удалим

In [239]:
comments.drop(columns ='Unnamed: 0', inplace = True)


Скорее всего классы будут несбаланированы. Заранее узнаем какое соотношение классов в исходных данных

In [240]:
(comments['toxic'].value_counts()) / len(comments) * 100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

Так как нужно всего лишь классифицировать токсичные комментарии c f1 не менее 0.75, считаю, что использовать BERT будет лишним. С этим должны справиться и более простые методы.

В исходном корпусе 160 тысяч записей. Кажется, это слишком много. Чтобы не перегружать код, возьмем случайную выборку из 40 000 записей

In [257]:
df = comments.sample(50000, random_state= 1).reset_index()

И сразу проверим, что исходное соотношение классов сохраняется

In [258]:
(df['toxic'].value_counts()) / len(df) * 100

0    89.754
1    10.246
Name: toxic, dtype: float64

### Предобработка текста

Так как BERT не используется, нужно провести полную подготовку текста.

In [259]:
#убираем специальные символы
df['text'] = df['text'].str.replace(r'[\n"]', '', regex=True)

#убираем пунктуацию
df['text'] = df['text'].str.replace(r'[^a-zA-Z]', ' ', regex=True) 

#приводим к нижнему регистру
df['text'] = df['text'].str.lower()

In [260]:
#Лемматизация
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


for i in notebook.tqdm(range(df.shape[0])):
    doc = nlp(df.loc[i,'text'])
    df.loc[i,'lemm']=" ".join([token.lemma_ for token in doc])


  0%|          | 0/50000 [00:00<?, ?it/s]

In [261]:
df['lemm'].head(2)

0    bite I that be all    cunt                                                                                                                                                                                                                                                    
1    actually   I m certain this will be the last break for the u s    also   the boiling rock be not confirm   so you should not count that in your calculation   but I agree it should be leave out until some official word be announce   the same go for the other book   rau  
Name: lemm, dtype: object

## Обучение

### Подготовка данных

Так как известно, что классы не сбалансированы, подготовим функцию для увеличения выборки редкого класса:

In [262]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = vstack([features_zeros] + [features_ones] * repeat)
    target_upsampled = pandas.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=1)
    return features_upsampled, target_upsampled

Теперь все готово для обучения

In [263]:
#Выделение признаков и таргета
X = df['lemm']
y = df['toxic']

In [264]:
#разделение на обучающую и тестовую выборки

X_train, X_valid,y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=1, stratify =y)
X_valid, X_test,y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=1, stratify =y_valid)

In [265]:
vector = TfidfVectorizer()
vector.fit(X_train)
X_train = vector.transform(X_train)
X_test = vector.transform(X_test)
X_valid = vector.transform(X_valid)

In [266]:
#увеличение частоты редкого класса

X_train, y_train = upsample(X_train, y_train, 4)

Для начала проверим, что получается для модели, которая все записи считает токсичными. Это будет полезно для оценки адекватности моделей

In [267]:
dummy_pred = DummyClassifier(strategy='constant',constant=1).fit(X_train,y_train).predict(X_valid)
f1_score(y_test,dummy_pred)

0.18594104308390022

Опробуем теперь несколько моделей - логистическую регрессию, случайный лес и CatBoost

### Логистическая регрессия

In [268]:
lr = LogisticRegression()
lr.fit(X_train,y_train)


0.7540322580645161

### Случайный лес

In [269]:
%%time
rfc = RandomForestClassifier(random_state=1)

rfc_parameters = {'max_depth'    : [100,200, 300],
                  'n_estimators' : [6,8,10]
                 }

grid_rfc = GridSearchCV(estimator=rfc, param_grid = rfc_parameters,  scoring='f1')
grid_rfc.fit(X_train, y_train);

CPU times: user 3min 50s, sys: 393 ms, total: 3min 50s
Wall time: 3min 51s


GridSearchCV(estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_depth': [100, 200, 300],
                         'n_estimators': [6, 8, 10]},
             scoring='f1')

### CatBoost

In [271]:
cbc = CatBoostClassifier(custom_metric='F1',silent = True, random_state =1, iterations = 100)
cbc.fit(X_train,y_train)

0.7520325203252034

### Сравнение моделей

In [274]:
models = [lr,rfc,cbc]
model_names = ['Logistic Regression', 'Random Forest', 'Cat Boost']
results = pandas.DataFrame(columns=['model','f1'])
for i in range(len(models)):
    model = models[i]
    name = model_names[i]
    pred = model.predict(X_valid)
    f1 = f1_score(y_valid,pred)
    results = results.append({'model': name,
                             'f1': f1}, 
                             ignore_index=True)  
results

,model,f1
0,Logistic Regression,0.754032
1,Random Forest,0.517845
2,Cat Boost,0.752033


Две модели из трех достигли требуемой f1, лучше всех справилась логистическая регрессия с f1 равной 0,754. Проверим ее окончательно на тестовой выборке:

In [276]:
pred_lr = lr.predict(X_test)
f1_score(y_test,pred_lr)

0.7590000000000001

## Выводы

- из исходной выборки случайным образом выделено 50 000 записей. Этого достаточно, чтобы добиться требуемого значения метрики и при этом не перегружать оперативную память
- обучено несколько моделей
- модель логистической регрессии дает f1 равное 0.76
- магазину Викишоп можно рекомендовать использовать полученную модель для поиска токсичных записей